In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
import matplotlib.pylab as plt
%matplotlib inline
import os
import datetime

/root/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [22]:
data = pd.read_csv('../data/sample_train.csv')

In [23]:
drop_feature = ['appIdAction','appIdInstall','interest3','interest4','kw3','topic3']
data.drop(drop_feature, axis=1,inplace=True)

In [24]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
print('Before dummification we have {} variables in train'.format(data.shape[1]))
train = pd.get_dummies(data, columns=one_hot_feature)
print('After dummification we have {} variables in train'.format(train.shape[1]))

Before dummification we have 27 variables in train
After dummification we have 1044 variables in train


In [ ]:
# data[['label', 'creativeId']].groupby(['creativeId']).apply(lambda x:x['label'].sum()).reset_index(name='creativeId_all')
# tmp = data[['label', 'creativeId', 'house']].groupby(['creativeId', 'house'])
# ['house', 'label'].agg(feaDict).reset_index()
# pd.merge(data, tmp, how='left', on='creativeId')
# tmp.apply(lambda x:x['label'].sum())
# count = tmp.apply(lambda x:x['label'].sum()).reset_index(name='creative_house')

In [25]:
# one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house','os','ct','marriageStatus','advertiserId','campaignId', 'creativeId',
#        'adCategoryId', 'productId', 'productType']
vector_feature= ['interest1','interest2', 'interest5', 'kw1', 'kw2', 'topic1', 'topic2']
# for feature in one_hot_feature:
#     try:
#         data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
#     except:
#         data[feature] = LabelEncoder().fit_transform(data[feature])
# train_x.shape,train_a.shape

In [61]:
# cv.fit(train_x['interest1'])
# train_a = cv.transform(train_x['interest1'])
# sparse.hstack((train_x, train_a))

TypeError: no supported conversion for types: (dtype('O'), dtype('int64'))

In [41]:
train_x = train[['creativeSize']]
cv=CountVectorizer(min_df=0.0009)
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(data[feature])
#     test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a))
#     test_x = sparse.hstack((test_x, test_a))
print('cv prepared !')

cv prepared !


In [48]:
train_x.shape,train.shape

((87988, 3860), (87988, 1037))

In [32]:
train.drop(vector_feature, axis=1, inplace=True)

In [52]:
train_x = sparse.hstack((train_x, train))

In [53]:
train_x.shape

(87988, 4897)

In [75]:
# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.3, random_state=2018)
# test=data[data.label==-1]
# test_id=test[['aid','uid']]
# test=test.drop('label',axis=1)
# enc = OneHotEncoder()
# train_x=train[['creativeSize']]
# # test_x=test[['creativeSize']]

# for feature in one_hot_feature:
#     enc.fit(data[feature].values.reshape(-1, 1))
#     train_a=enc.transform(train[feature].values.reshape(-1, 1))
# #     test_a = enc.transform(test[feature].values.reshape(-1, 1))
#     train_x= sparse.hstack((train_x, train_a))
#     break
# #     test_x = sparse.hstack((test_x, test_a))
# print('one-hot prepared !')

one-hot prepared !


In [91]:
enc.fit(data[feature].values.reshape(-1, 1))

array([162, 273, 162, ...,  58,   7,  45])

In [85]:
sparse.hstack((train, train_a))

TypeError: no supported conversion for types: (dtype('O'), dtype('float64'))

In [32]:
train_y=train.pop('label')

KeyError: 'label'

In [7]:
train_x.shape,train_y.shape

((87988, 5497), (87988,))

In [8]:
train_y.to_csv('../data/train_y.csv', index=False)
# test_y.to_csv('../data/test_y.csv', index=False)

In [9]:
sparse.save_npz('../data/train_gen.npz', train_x)
# sparse.save_npz('../data/test_gen.npz', test_x)

In [10]:
# data = pd.read_csv('../data/sample_train.csv')
train_x = sparse.load_npz('../data/train_gen.npz')
# test_x = sparse.load_npz('../data/test_gen.npz')
train_y = pd.read_csv('../data/train_y.csv', header=None)
# test_y = pd.read_csv('../data/test_y.csv')

In [20]:
def func(x1, x2):
    return str(x1)+'_'+str(x2)

def func2(x1, x2, x3):
    return str(x1)+'_'+str(x2)+"_"+str(x3)

In [ ]:
value = list(map(func, df['1001'], df['4001']))
dummies = pd.get_dummies(value, prefix='1001--4001')
print(dummies)
df = pd.concat([df, dummies], axis=1)

In [3]:
clf = XGBClassifier(    
                        n_estimators=1000,
                        max_depth=7,
                        objective="binary:logistic",
                        learning_rate=0.1, 
                        subsample=.8,
                        min_child_weight=5,
                        colsample_bytree=.9,
                        #scale_pos_weight=1.6,
                        gamma=0,
                        #reg_alpha=8,
                        #reg_lambda=1.3,
                        n_jobs=-1,
                        scale_pos_weight=1,
                        seed=2018
                     )

In [ ]:
params = {
    'n_estimators' : np.arange(1000, 8000, 1000),
    'num_leaves' : np.arange(30, 45, 2),
    'min_child_weight' : np.arange(40, 65, 5),
    'reg_alpha' : [0.2, 0.4, 0.8, 1],
    'reg_lambda' : [0.2, 0.4, 0.8, 1],
}

In [19]:
np.ravel(train_y)

array([0, 0, 0, ..., 0, 0, 0])

In [55]:
def modelfit(alg, x_train, y_train, useTrainCV=False, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(x_train, label=y_train)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=50, show_stdv=True)
        alg.set_params(n_estimators=cvresult.shape[0])
        print ("BEST Tree Nums is %s" % (cvresult.shape[0]))
    #Fit the algorithm on the data
    alg.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=early_stopping_rounds)

    #Predict training set:
    dtrain_predictions = alg.predict(x_train)
    dtrain_predprob = alg.predict_proba(x_train)[:,1]

    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_train.values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob))
    feat_imp = pd.Series(alg.get_booster().get_fscore()).sort_values(ascending=False)
    feat_imp.to_csv('../data/fea_imp.csv') 
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')
    return alg

In [56]:
print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
clf = modelfit(clf, train_x, train_y)
print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2018-05-15 11:16:02
[0]	validation_0-auc:1
Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:1
[2]	validation_0-auc:1
[3]	validation_0-auc:1
[4]	validation_0-auc:1
[5]	validation_0-auc:1
[6]	validation_0-auc:1
[7]	validation_0-auc:1
[8]	validation_0-auc:1
[9]	validation_0-auc:1
[10]	validation_0-auc:1
[11]	validation_0-auc:1
[12]	validation_0-auc:1
[13]	validation_0-auc:1
[14]	validation_0-auc:1
[15]	validation_0-auc:1
[16]	validation_0-auc:1
[17]	validation_0-auc:1
[18]	validation_0-auc:1
[19]	validation_0-auc:1
[20]	validation_0-auc:1
[21]	validation_0-auc:1
[22]	validation_0-auc:1
[23]	validation_0-auc:1
[24]	validation_0-auc:1
[25]	validation_0-auc:1
[26]	validation_0-auc:1
[27]	validation_0-auc:1
[28]	validation_0-auc:1
[29]	validation_0-auc:1
[30]	validation_0-auc:1
[31]	validation_0-auc:1
[32]	validation_0-auc:1
[33]	validation_0-auc:1
[34]	validation_0-auc:1
[35]	validation_0-auc:1
[36]	validation_0-auc:1
[37]	validation_0-auc:1
[38]	validatio

/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:



Model Report
Accuracy : 1
AUC Score (Train): 1.000000
2018-05-15 11:16:10


In [65]:
fea_imp = pd.read_csv('../data/fea_imp.csv')
fea_imp

,f0,172
0,f455,65
1,f24163,61
2,f549,57
3,f480,53
4,f465,52
5,f459,47
6,f24338,45
7,f460,44
8,f24345,43
9,f24187,42


In [2]:
train_x = sparse.load_npz('../data/train_gen_520.npz')
test_x = sparse.load_npz('../data/test_gen_520.npz')
train_y = pd.read_csv('../data/train_y_520.csv', header=None)
train_id=pd.read_csv('../data/train.csv')[['aid','uid']]
predict_id=pd.read_csv('../data/test2.csv')[['aid','uid']]

((8798814, 419545), (2265879, 419545))

In [8]:
from sklearn.cross_validation import StratifiedKFold

In [14]:
test_x[1]

TypeError: 'coo_matrix' object does not support indexing

In [12]:
X_sparse = test_x.tocsr()
# y= train_y
# X_train = {}
# Y_train = {}

# skf = StratifiedKFold(X_sparse, n_folds=5, shuffle=True, random_state=12345)
# i=0
# for train_index, test_index in skf.split(X_sparse):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train[i], X_test[i] = X[train_index], X[test_index]
#     y_train[i], y_test[i] = y[train_index], y[test_index]
#     i +=1

<1x419545 sparse matrix of type '<class 'numpy.float64'>'
	with 60 stored elements in Compressed Sparse Row format>

In [ ]:
param_test1 = {
  'max_depth':[6,7,8]
}
gsearch1 = GridSearchCV(estimator = model, 
 param_grid = param_test1,     scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch1.fit(train_x, train_y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [17]:
test_x_csr = test_x.tocsr()
train_x_csr = train_x.tocsr()

In [19]:
train_y.shape

(8798814, 1)

In [18]:
train_x_csr.shape, test_x_csr.shape

((8798814, 419545), (2265879, 419545))

In [15]:
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)
OPTIMIZE_ROUNDS=True

In [16]:
for i, (train_index, test_index) in enumerate(kf.split(X_sparse)):
    print (train_index, test_index)

[      1       2       4 ... 2265876 2265877 2265878] [      0       3      13 ... 2265840 2265852 2265864]
[      0       1       3 ... 2265876 2265877 2265878] [      2       5       7 ... 2265870 2265871 2265873]
[      0       1       2 ... 2265875 2265876 2265877] [      6      33      39 ... 2265868 2265874 2265878]
[      0       1       2 ... 2265875 2265876 2265878] [      4      11      17 ... 2265869 2265872 2265877]
[      0       2       3 ... 2265874 2265877 2265878] [      1      22      25 ... 2265867 2265875 2265876]


In [ ]:
train_x

In [20]:
pd.read_csv('../data/train_y_518.csv', header=None).shape

(8798814, 1)

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(train_x)):

    # Create data for this fold
    y_train, y_valid = train_y.iloc[train_index].copy(), train_y.iloc[test_index].copy()
    X_train, X_valid = train_x_csr[train_index.tolist(), :].copy(), train_x_csr[test_index.tolist(), :].copy()
    X_test = test_x_csr.copy()
    print ("\nFold ", i)

    if OPTIMIZE_ROUNDS:
        eval_set = [(X_valid, y_valid)]
        fit_model = model.fit(X_train, y_train,
                              eval_set=eval_set,
                              eval_metric='auc',
                              early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                              verbose=False
                              )
        print ("  Best N trees = ", model.best_ntree_limit)
        print ("  Best AUC = ", model.best_score)
    else:
        fit_model = model.fit(X_train, y_train)

    pred = fit_model.predict_proba(X_valid)[:, 1]
#     pred_encode = pred_proc_encode(pred)
#     print (classification_report(y_valid, pred_encode))
    print (fit_model.best_score_[ 'valid_1']['auc'])
    y_valid_pred.iloc[test_index] = pred

    # Accumulate test set predictions
    y_test_pred += fit_model.predict_proba(X_test)[:, 1]

    del X_test, X_train, X_valid, y_train

y_test_pred /= K  # Average test set predictions

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)

In [ ]:
y_test_pred /= K  # Average test set predictions
fit_model.save_model('../data/xgb.model')
y_pred = pred_proc_encode(y_test_pred)
y_pred_cv = pred_proc_encode(y_valid_pred)

print "\nreport for full training set:", classification_report(y, y_pred_cv, target_names=['negetive','positive'])
                                                                                           


submit = pd.concat([test_id, y_pred], axis=1)

submit.to_csv(data_path+'xgb_result_{}_test.csv'.format(code), index=False)